# Handwritten Character Recognition

In [48]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu,linear
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

done


## The Data
The data was taken from [this](https://www.kaggle.com/datasets/sachinpatel21/az-handwritten-alphabets-in-csv-format) Kaggle dataset, which provides a csv format of the original data from the [NIST](https://www.nist.gov/srd/nist-special-database-19) and MNIST datasets. 
<br>
The data represents 372451 images drawn on a 28x28 grid. The 1st column contains the classification label (0->25 representing A->Z), and the rest of the 784 columns contain the input features (numbers from 0->255 represnting the color degree of the pixel).

In [50]:
# Open the CSV file
with open('A_Z handwritten Data.csv', 'r') as file:
    reader = csv.reader(file)
    data = []
    for i, row in enumerate(reader):
        if i % 50000 == 0:
            print(f'iteration: {i}')
        data.append(row)
    data = np.array(data)

# Get the first column and store it in a numpy array
y = np.array(data[:, 0], dtype=float)

# Get the rest of the columns and store it in a numpy matrix
X = np.matrix(data[:, 1:], dtype=float)

print(y.shape)
print(X.shape)

record: 0
record: 10000
record: 20000
record: 30000
record: 40000
record: 50000
record: 60000
record: 70000
record: 80000
record: 90000
record: 100000
record: 110000
record: 120000
record: 130000
record: 140000
record: 150000
record: 160000
record: 170000
record: 180000
record: 190000
record: 200000
record: 210000
record: 220000
record: 230000
record: 240000
record: 250000
record: 260000
record: 270000
record: 280000
record: 290000
record: 300000
record: 310000
record: 320000
record: 330000
record: 340000
record: 350000
record: 360000
record: 370000
(372451,)
(372451, 784)
25.0


## Splitting our Data
We split our dataset into training and test sets so we can later compare the missclassifications to check for the presence of bias and variance issues. 
<br>
Training Set: 75%
<br>
Test Set: 25%

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=1)

## The Model
### Layers
I tested different models with different layer strctures, 100-100-100, 500-200-100, 784-784-784 etc. Increasing the number of nodes per layer didnt seem to help reduce the cost/loss much beyond 100 layers. The model I ended up staying with was 100-100-100-100 (4 layers of 100 nodes and then a final layer of 26 nodes).
### Regulaizer
Using a regulaizer was causing the model to underfit the data, and the absence of one wasn't causing an overfitting issue, so ended up not using one.
### Loss & Optimizer
Used the Sparse Categorical Crossentropy loss function because this is a multi-class classification problem along with the setting to let tensorflow know to use the sigmoid function since the layer is set to a linear activation for numerical accuracy.
<br>
Used the Adam optimizer with a small learning rate of 0.0001 since larger values seemed to cause gradient descent to not work properly.

In [71]:
model = Sequential([
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(26, activation='linear')
])

In [72]:
model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(learning_rate=0.0001)
)

In [73]:
model.fit(X_train, y_train, epochs=30)

Epoch 1/30
8730/8730 [==============================] - 39s 4ms/step - loss: 1.4177
Epoch 2/30
8730/8730 [==============================] - 36s 4ms/step - loss: 0.3008
Epoch 3/30
8730/8730 [==============================] - 36s 4ms/step - loss: 0.2031
Epoch 4/30
8730/8730 [==============================] - 37s 4ms/step - loss: 0.1575
Epoch 5/30
8730/8730 [==============================] - 37s 4ms/step - loss: 0.1307
Epoch 6/30
8730/8730 [==============================] - 37s 4ms/step - loss: 0.1108
Epoch 7/30
8730/8730 [==============================] - 37s 4ms/step - loss: 0.0961
Epoch 8/30
8730/8730 [==============================] - 37s 4ms/step - loss: 0.0843
Epoch 9/30
8730/8730 [==============================] - 39s 4ms/step - loss: 0.0753
Epoch 10/30
8730/8730 [==============================] - 40s 5ms/step - loss: 0.0673
Epoch 11/30
8730/8730 [==============================] - 38s 4ms/step - loss: 0.0603
Epoch 12/30
8730/8730 [==============================] - 38s 4ms/step - lo

## Testing our Model
To test our model, we can simply loop over our dataset and see how many values it correctly classifies (or miss-classifies for that matter). We do the same for our test set so we can compare these values and see if our model generalizes well to data it has not seen before.

In [74]:
yhat = model.predict(X_train)
correct = 0
for i in range(len(y_train)):
    if y_train[i] == np.argmax(tf.nn.softmax(yhat[i])):
        correct += 1
print('accuracy with trainingt set: ' + str(correct/len(y_train)))

yhat = model.predict(X_test)
correct = 0
for i in range(len(y_test)):
    if y_test[i] == np.argmax(tf.nn.softmax(yhat[i])):
        correct += 1
print('accuracy with test set: ' + str(correct/len(y_test)))

8730/8730 [==============================] - 20s 2ms/step
accuracy with trainingt set: 0.9906743801416206
2910/2910 [==============================] - 6s 2ms/step
accuracy with test set: 0.9783918464661218


## Conclusion
On our training set, the model predicted ~ 99.07% of the labels correctly. On our test set, our model predicted ~ 97.84% of the labels correctly. The model does seem to generalize out to unseen data pretty well.

In [75]:
model.save('my_model')

INFO:tensorflow:Assets written to: my_model/assets


INFO:tensorflow:Assets written to: my_model/assets


## Error Analysis
We can run some code on our test set predictions to check if there is a pattern on letters that our model has trouble predicting. We can use this information to further tune our model in the future to get better results.

In [92]:
incorrect_count = [0] * 26
for i in range(len(y_test)):
    if y_test[i] != np.argmax(tf.nn.softmax(yhat[i])):
        incorrect_count[int(y_test[i])] += 1
for i in range(26):
    print(f'{i}: {incorrect_count[i]}')


0: 71
1: 75
2: 71
3: 65
4: 88
5: 6
6: 46
7: 153
8: 11
9: 65
10: 58
11: 55
12: 53
13: 42
14: 333
15: 39
16: 60
17: 105
18: 80
19: 60
20: 119
21: 14
22: 192
23: 37
24: 90
25: 24


From the looks of it, the missclassifications are split pretty evenly among all the letters except for a few anomalies. 
<br>
It has the most issues with predicting the letters 'O', 'W', and 'H' in with 333, 192 & 153 missclassificataions respectively. Note the average for each letter seems to be around ~ 60.
<br>
On the other hand, it almost always correctly classified 'F', 'I', and 'V', with only 6, 11, and 14 missclassifications.